# Filtering for Good Images
### ECOSTRESS Tutorials
###### This code is best suited for use when you have a folder of files that you would like to be automatically filtered to get just the good images. In this case, the good images are ones that have been georeferenced, have minimal cloud cover, and have mostly good quality (not null) pixels. In addition to the files you want to filter, the input directory should also have the images' associated QC files and a statistics CSV. This can all be downloaded from the AppEEARS website. 

### Import the Necessary Libraries

In [11]:
import os
from os import makedirs
import pandas as pd
import rasterio
import numpy as np

### Set the Directories

In [12]:
#Replace this with the path to the folder where the downloaded files are kept, wrapped in quotes
input_directory = r"Replace_this_text_with_folder_path"
    #Folder should include files of your layer of interest (LST, Emissivity, etc.), the associated QC files,
    #and one statistics CSV file that is associated with this download
#Replace this with the path to the folder where you want the outputs to go, wrapped in quotes
output_directory = r"Replace_this_text_with_folder_path"
#This line makes sure that the output directory exists
makedirs(input_directory, exist_ok=True)

#The threshold at which you want to decide to accept a ECOSTRESS product based on the quality of pixels.
    #Ex: A threshold of 0.75 means that at least 75% of the pixels have to be good quality.
    #You can change this if you want. A higher number means more strict quality requirements. 
thresh = 0.75 #has to be between 0 and 1


### Create Lists of all the Different File Types

In [13]:
scene_list = [] #This will contain all the paths for your layer of interest (LST, Emissivity, etc.), which we will call scenes
QC_list = [] #This will contain the paths to the QC files associated with your layer of interest
csv_count = 0 #Counts the number of CSV files in your directory, starting at 0

#Checks if the input directory is empty, and if it is, it will give you an error
if len(os.listdir(input_directory)) == 0  : 
    raise(ValueError("The directory you typed is empty, please check it."))

#loop through each file in the input directory
for file in os.listdir(input_directory):
    #Get each file's path by joining the input directory with the file name
    file_path = os.path.join(input_directory,file)
    #Check if the file ends in .csv ...
    if file.endswith('csv'): 
        #If it does, increase the CSV count list by 1
        csv_count +=1
        #read the CSV into the variable (pandas DataFrame) called eco_stats
        eco_stats = pd.read_csv(file_path)
    #Check if the file ends in .tif and contains QC
    elif file.endswith('.tif') and file.__contains__('QC'):
        #If it does, add it to the list of QC files
        QC_list.append(file)
    #Check all other files to see if they end in .tif (these are the layers of interest or scenes)
    elif file.endswith('.tif'):
        #If they do, add it to the scene list 
        scene_list.append(file)

#Creates a set of warnings that will appear if any of your files are missing from the directory
#Checks if there is exactly one CSV
if csv_count != 1:
    raise(ValueError("The directory contains an abnormal number of statistics CSVs, there should only be exactly one."))      
#Checks if there are QC files
if not QC_list: 
    raise(ValueError("The directory doesn't contain any QC files, please check it."))
#Checks if there are scenes
if not scene_list: 
    raise(ValueError("The directory doesn't contain any scenes, please check it."))

### Process QC (Quality Control) Files into QF (Quality Flag) Files

In [14]:
# QC files processing to render them easily readable by an user and for validity by python
#Creates a path for a directory called QF in the output directory
QF_directory = os.path.join(output_directory,'QF')
#Checks if this directory exists...
if not os.path.exists(QF_directory):
    #... and if it doesn't, create it
    os.mkdir(QF_directory)

#Loop through each file in the QC list
for qc in QC_list :
    #Get the file's path by joining the input directory with the QC file name
    qc_file = os.path.join(input_directory,qc)
    #Opens the QC_file
    with rasterio.open(qc_file,'r') as f_qc:
        qc_img = f_qc.read((1)) #Read the first band of the QC file. They are coded in 16 bits
        qc_img[qc_img==-99999] = -1  #Nodata values are read as -99999, we change it to -1 so that the last two bits appear as 11 (which means pixel not produced) and will be masked out in the end
        qc_img_2 = qc_img & 0b11 #Select only the last two bits that tell us what the quality of the pixel is
        out_meta = f_qc.meta.copy() #Copies the metadata from the original QC file
    
    #Creates the QF file path by joining the QF directory with the file name, but change QC to QF in the file name
    file_qf = os.path.join(QF_directory,qc.replace('QC','QF'))
    #Opens the new QF file and adds the copied metadata
    with rasterio.open(file_qf,'w',**out_meta) as dst: 
        #Writes the data into the QF new file
        dst.write(qc_img_2,1)

### Filters for Good Images
#### If bad, filters them into separate categories depending on why they are bad. The categories are bad georeference. cloudy, or no data. 

In [15]:
#Creates a function called copy_file that we will use later to copy files from the input folder into the correct output folder
def copy_file(source, destination):
    with open(source, 'rb') as src, open(destination, 'wb') as dst:
        dst.write(src.read())

###Checking for bad georeferencing###
#loops through each file in the scene list
for scene in scene_list : 
    #Find the row in the eco_stats CSV where the file name column matches the scene name
    matching_scene = eco_stats.loc[eco_stats['File Name']==scene[:-4].replace('.','_')]
    #Gets the value from the coulmn titled "Orbit Correction Preformed" for that row
    orbit_status = matching_scene['Orbit Correction Performed'].item() 
    #If the orbit correction hasn't been performed ...
    if not orbit_status == 1:
        #... discard the file into a bag georeference folder
        bad_geo_dir = os.path.join(output_directory,'Bad georeference')
        #If the directory for the bad georeferenced images does not exist ...
        if not os.path.exists(bad_geo_dir):
            #... create it
            os.mkdir(bad_geo_dir)
        #Gets the path to the original file from the input directory
        source_path = os.path.join(input_directory,scene)
        #Creates a path for the file in the bag georeferenced directory
        dst_path = os.path.join(bad_geo_dir,scene)
        #Copies the file from the input directory to the bad georeferenced directory
        copy_file(source_path, dst_path)
    
    #If the orbit correction has been preformed ...
    else:
        #... get the dataset value from the matching row in the CSV
        scene_type = matching_scene['Dataset'].item()
        #Creates a path for the masked file by replacing the last three characters of the scene type with QF_
        mask_file_for_scene =os.path.join(QF_directory,scene.replace(scene_type[-3:]+'_','QF_'))        
        
        #Opens the masked file with rasterio
        with rasterio.open(mask_file_for_scene,'r') as mask:
            #Reads the first band of the file
            mask_array = mask.read(1)
            #Stores the size of the first band into the variable mask sz
            mask_size = mask_array.size
       
###Checking for other qulaity flags (clouds and no data)###
        #Checks if the proportion of the good quality pixels is greater than the threashold 
            #If it is, we accept it as good. If not, we look into why  
        if np.count_nonzero(mask_array==0) + np.count_nonzero(mask_array==1)>thresh*mask_size:
            #Creates a path to the good folder in the output directory
            good_dir = os.path.join(output_directory, 'Good')
            #If the good directory does not exist ...
            if not os.path.exists(good_dir):
                #... create it
                os.makedirs(good_dir)
            #Gets the path to the original file from the input directory
            source_path = os.path.join(input_directory, scene)
            #Creates a path for the file in the good directory
            dst_path = os.path.join(good_dir, scene)
            #Copies the file from the input directory to the good directory
            copy_file(source_path, dst_path)
        else:
            #Checks if there are more cloudy pixels than no data pixels. If the main reason for the image to be disregarded is clouds, we move it into the Cloudy folder
            if np.count_nonzero(mask_array==2) > np.count_nonzero(mask_array==3):
                #Creates a path to the cloudy folder in the output directory
                cloudy_dir = os.path.join(output_directory,'Cloudy')
                #If the cloudy directory does not exist ...
                if not os.path.exists(cloudy_dir) :
                    #... create it
                    os.mkdir(cloudy_dir)
                #Gets the path to the original file from the input directory
                source_path = os.path.join(input_directory,scene)
                #Creates a path for the file in the cloudy directory
                dst_path = os.path.join(cloudy_dir,scene)
                #Copies the file from the input directory to the cloudy directory
                copy_file(source_path, dst_path)

            #If there are more no data pixels, move it into the no data output folder
            else : 
                #Creates a path to the no data in the output directory
                nodata_dir = os.path.join(output_directory,'Nodata')
                #If the no data directory does not exist ...
                if not os.path.exists(nodata_dir):
                    #... create it
                    os.mkdir(nodata_dir)
                #Gets the path to the original file from the input directory
                source_path = os.path.join(input_directory,scene)
                #Creates a path for the file in the no data directory
                dst_path = os.path.join(nodata_dir,scene)
                #Copies the file from the input directory to the no data directory
                copy_file(source_path, dst_path)